In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt

In [49]:
deals = pd.read_excel('Deals (Done).xlsx', dtype={'Id': str, 'Contact Name': str})
deals.head()

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Page,Campaign,SLA,Content,...,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,Level of Deutsch
0,5805028000056864695,Ben Hall,NaN,NaN,New Lead,NaN,/eng/test,03.07.23women,NaN,v16,...,NaN,NaN,21.06.2024 15:30,NaN,NaN,NaN,NaN,5805028000056849495,NaN,NaN
1,5805028000056859489,Ulysses Adams,NaN,NaN,New Lead,NaN,/at-eng,NaN,NaN,NaN,...,Web Developer,Morning,21.06.2024 15:23,6.0,NaN,0,2000,5805028000056834471,NaN,NaN
2,5805028000056832357,Ulysses Adams,21.06.2024,D - Non Target,Lost,Non target,/at-eng,engwien_AT,00:26:43,b1-at,...,NaN,NaN,21.06.2024 14:45,NaN,NaN,NaN,NaN,5805028000056854421,NaN,NaN
3,5805028000056824246,Eva Kent,21.06.2024,E - Non Qualified,Lost,Invalid number,/eng,04.07.23recentlymoved_DE,01:00:04,bloggersvideo14com,...,NaN,NaN,21.06.2024 13:32,NaN,NaN,NaN,NaN,5805028000056889351,NaN,NaN
4,5805028000056873292,Ben Hall,21.06.2024,D - Non Target,Lost,Non target,/eng,discovery_DE,00:53:12,website,...,NaN,NaN,21.06.2024 13:21,NaN,NaN,NaN,NaN,5805028000056876176,NaN,NaN


In [50]:
#Convert the data types for the updated data
deals['Created Time'] = pd.to_datetime(deals['Created Time'],format='%d.%m.%Y %H:%M', dayfirst=True, errors='raise')
deals['Closing Date'] = pd.to_datetime(deals['Closing Date'],format='%d.%m.%Y', dayfirst=True, errors='raise')
deals[['Course duration', 'Months of study']] = deals[['Course duration',  'Months of study']].astype('Int8')
deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21592 entries, 0 to 21591
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Id                   21592 non-null  object        
 1   Deal Owner Name      21563 non-null  object        
 2   Closing Date         14645 non-null  datetime64[ns]
 3   Quality              19339 non-null  object        
 4   Stage                21592 non-null  object        
 5   Lost Reason          16123 non-null  object        
 6   Page                 21592 non-null  object        
 7   Campaign             16067 non-null  object        
 8   SLA                  15533 non-null  object        
 9   Content              14147 non-null  object        
 10  Term                 12454 non-null  object        
 11  Source               21592 non-null  object        
 12  Payment Type         496 non-null    object        
 13  Product              3592 non-n

In [51]:
#Dealing with Duplicate status in the Lost Reason column

In [52]:
# Load the Excel file and ensure 'Contact Name' is treated as a clean string
#deals = pd.read_excel('Deals (Done).xlsx', dtype={'Id': str})
#deals['Contact Name'] = deals['Contact Name'].astype(str).str.strip()

# Store the original number of rows for comparison later
original_shape = deals.shape

# Identify contact names that appear more than once in the dataset
contact_name_ids_more_than_one = (
    deals['Contact Name']
    .value_counts()
    .loc[lambda x: x > 1]
    .index
    .tolist()
)

# Select only deals marked as 'Lost' with the reason 'Duplicate' and a valid contact name
filtered_df = deals[
    (deals['Stage'] == 'Lost') &
    (deals['Lost Reason'] == 'Duplicate') &
    (deals['Contact Name'].notna())
]

# Keep only rows with contact names known to be duplicated
duplicates_contact_name = filtered_df[
    filtered_df['Contact Name'].isin(contact_name_ids_more_than_one)
].copy()

# Ensure 'Created Time' is properly parsed as datetime
deals['Created Time'] = pd.to_datetime(deals['Created Time'], errors='coerce')
duplicates_contact_name['Created Time'] = pd.to_datetime(
    duplicates_contact_name['Created Time'], errors='coerce'
)

# Sort duplicates chronologically so we can retrieve first/last values consistently
sorted_df = duplicates_contact_name.sort_values('Created Time')

# Aggregate duplicated rows by 'Contact Name'
# - For the 'Source' column: keep the first (oldest) value
# - For all other columns: keep the last non-zero, if none then last non-null, otherwise NaN
aggregated_contacts = sorted_df.groupby('Contact Name').agg({
    'Source': lambda x: x.iloc[0],  # Take the first (oldest) value for 'Source'
    
    # Apply custom aggregation for all other columns
    **{
        col: lambda x: (
            x.dropna()[x.dropna() != 0].iloc[-1]     # Take last non-zero value
            if not x.dropna()[x.dropna() != 0].empty
            else x.dropna().iloc[-1]                 # Otherwise last non-null
            if not x.dropna().empty
            else np.nan                              # Or NaN if nothing valid
        )
        for col in sorted_df.columns if col not in ['Contact Name', 'Source']
    }
}).reset_index()

# Remove original duplicate rows from the dataset
deals_cleaned = deals.drop(index=duplicates_contact_name.index)

# Append the aggregated deduplicated rows back into the cleaned dataset
deals = pd.concat([deals_cleaned, aggregated_contacts], ignore_index=True)

# Compute and report the number of rows removed
final_shape = deals.shape
rows_removed = original_shape[0] - final_shape[0]

print(f"Original rows: {original_shape[0]}")
print(f"Final rows: {final_shape[0]}")
print(f"Rows removed: {rows_removed}")

Original rows: 21592
Final rows: 21411
Rows removed: 181


In [53]:
#Quality column
deals['Quality'].value_counts()

Quality
E - Non Qualified    7486
D - Non Target       6228
C - Low              3452
B - Medium           1560
A - High              430
F                       2
Name: count, dtype: int64

In [54]:
deals['Quality'] = deals['Quality'].replace('F', 'E - Non Qualified')
deals['Quality'] = deals['Quality'].fillna('E - Non Qualified')
deals['Quality'] = deals['Quality'].astype(str)
deals['Quality'].value_counts()

Quality
E - Non Qualified    9741
D - Non Target       6228
C - Low              3452
B - Medium           1560
A - High              430
Name: count, dtype: int64

In [55]:
# Filter empty rows
print("Rows with all values missing:", deals.isnull().all(axis=1).sum())

Rows with all values missing: 0


In [56]:
# Display unique values in the 'Initial Amount Paid' and 'Offer Total Amount' columns
print(deals['Initial Amount Paid'].unique())
print(deals['Offer Total Amount'].unique())

[nan 0 1000 '€ 3.500,00' 500 100 4500 300 200 2000 11000 4000 3000 3500
 11500 1200 1500 1 5000 600 700 350 9 400 450]
[nan 2000 9000 11000 3500 4500 '€ 2.900,00' 6500 4000 3000 10000 2500 5000
 11500 1 1000 1200 0 1500 '€ 11398,00' 11111 6000]


In [57]:
# Function to clean currency columns by removing symbols and converting to float
def clean_currency_col(df, col_name):
    df[col_name] = (
        df[col_name]
            # Remove the Euro symbol (€)
            .replace(r'[€]', '', regex=True)
            # Remove all whitespace characters
            .replace(r'\s+', '', regex=True)
            # Remove dots used as thousand separators (e.g., "3.500,00" → "3500,00")
            .replace(r'\.', '', regex=True)
            # Replace comma with dot to match decimal format (e.g., "3500,00" → "3500.00")
            .replace(r',', '.', regex=True)
            # Convert the cleaned strings to float numbers
            .astype(float)
    )
    return df

# Apply cleaning to both currency-related columns
deals = clean_currency_col(deals, 'Initial Amount Paid')
deals = clean_currency_col(deals, 'Offer Total Amount')

# Create a boolean mask where the initial amount paid is greater than the offer total
mask = deals['Initial Amount Paid'] > deals['Offer Total Amount']

# Swap the values in the affected rows to maintain logical consistency
# This corrects cases where someone accidentally entered values in the wrong columns
deals.loc[mask, ['Initial Amount Paid', 'Offer Total Amount']] = (
    deals.loc[mask, ['Offer Total Amount', 'Initial Amount Paid']].values
)

print("Rows where 'Initial Amount Paid' was greater than 'Offer Total Amount' and were corrected:")
print(deals[mask][['Initial Amount Paid', 'Offer Total Amount']])

# Show the cleaned columns
print("\nSample of cleaned and corrected columns:")
print(deals[['Initial Amount Paid', 'Offer Total Amount']].head(5))

Rows where 'Initial Amount Paid' was greater than 'Offer Total Amount' and were corrected:
       Initial Amount Paid  Offer Total Amount
1243                2900.0              3000.0
1351               11000.0             11500.0
1366               11000.0             11500.0
1397               11000.0             11500.0
1409               11000.0             11500.0
1441                2900.0              3000.0
1658               11000.0             11500.0
1920                2500.0              3000.0
2012               11000.0             11500.0
2272               11000.0             11500.0
2385                3000.0              3500.0
2700               11000.0             11500.0
3012               11000.0             11500.0
3189               11000.0             11500.0
3301               11000.0             11500.0
3795               11000.0             11500.0
4344               11000.0             11500.0
4457               11000.0             11500.0
4720            

In [58]:
deals['Initial Amount Paid'] = deals['Initial Amount Paid'].fillna('Unknown')
deals['Offer Total Amount'] = deals['Offer Total Amount'].fillna('Unknown')

In [59]:
# Function to convert time-like values to seconds
def convert_to_seconds(x):
    if pd.isna(x):
        return np.nan
    elif isinstance(x, datetime.time):
        # Convert time object to total seconds
        return x.hour * 3600 + x.minute * 60 + x.second
    elif isinstance(x, datetime.timedelta):
        # Convert timedelta object to total seconds
        return x.total_seconds()

# Apply conversion to a new column 'SLA Seconds'
deals['SLA in Seconds'] = deals["SLA"].apply(convert_to_seconds).round(2)

# Display the count of unique values in the 'SLA Seconds' column
deals['SLA in Seconds'].value_counts()

SLA in Seconds
669.0      6
611.0      6
646.0      6
1224.0     5
1063.0     5
          ..
7686.0     1
16217.0    1
28592.0    1
12023.0    1
52672.0    1
Name: count, Length: 13334, dtype: int64

In [60]:
# Drop the original 'SLA' column as it's now converted
deals.drop(columns=['SLA'], inplace=True)
deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21411 entries, 0 to 21410
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Id                   21411 non-null  object        
 1   Deal Owner Name      21382 non-null  object        
 2   Closing Date         14495 non-null  datetime64[ns]
 3   Quality              21411 non-null  object        
 4   Stage                21411 non-null  object        
 5   Lost Reason          15942 non-null  object        
 6   Page                 21411 non-null  object        
 7   Campaign             16006 non-null  object        
 8   Content              14101 non-null  object        
 9   Term                 12424 non-null  object        
 10  Source               21411 non-null  object        
 11  Payment Type         496 non-null    object        
 12  Product              3592 non-null   object        
 13  Education Type       3299 non-n

In [61]:
# Fill missing values in the 'City' column based on the most frequent value (mode)
# for each contact in the 'Contact Name' column

mode_values = deals.groupby('Contact Name')['City'] \
    .agg(lambda x: x.mode()[0] if not x.mode().empty else None)

# Map the most frequent city for each contact back to the 'City' column
deals['City'] = deals['Contact Name'].map(mode_values)

# Fill missing values in the 'Level of Deutsch' column based on the most frequent value per contact
mode_values = deals.groupby('Contact Name')['Level of Deutsch'] \
    .agg(lambda x: x.mode()[0] if not x.mode().empty else None)

# Map the most frequent language level for each contact back to the 'Level of Deutsch' column
deals['Level of Deutsch'] = deals['Contact Name'].map(mode_values)

print("Number of missing values after filling:")
print(deals[['City', 'Level of Deutsch']].isna().sum())

Number of missing values after filling:
City                18146
Level of Deutsch    19756
dtype: int64


In [62]:
replace_dict = {
    'в1': 'B1', 'б1': 'B1', 'b1': 'B1', 'B1': 'B1', 'в1-в2': 'B1',
    'B2': 'B2', 'C2': 'C2', 'с1': 'C1', 'Б1': 'B1', 'а2': 'A2', 'а1': 'A1',
    'а0': 'A0','б2': 'B2', 'Б2': 'B2', 'В1': 'B1', 'А2': 'A2',
    'B1 будет в феврале 2025': 'A2',
    'Detmold, Paulinenstraße 95, 32756': 'Not defined',
    'Сам оценивает на B2, 13 лет живет в Германии': 'B2',
    'в2': 'B2', 'В1-В2': 'B1', 'Б1 ( ждет Б2)': 'B1', 'В1': 'B1',
    'lэкзамен - 6 июля на В1. курсы вечером (но уверенно говорит на B1)': 'B1',
    'Гражданка Германии уже год в Германии Учит немецкий и в сентябре b1 через гос-во проходит, а не через ДЖЦ, вечером учится 3 р в неделю с 18 до 21': 'B1',
    '-': 'A0', 'А2 ( Б1 в июне)': 'A2', 'B1 в процессе обучения': 'A2','ЯЗ: нем В1 был экз 03.05 повтор и сейчас ждет результаты. Технический англ был. А1 сейчас. ОБР: 2 во информационные и комп сети - инженер системоте': 'B1',
    'В1 в сентябре': 'A2', 'Нет': 'Not defined', 'С1': 'C1', 0: 'A0',
    'Ждем B1': 'B1', 'А1 сертиф, но по факту А2': 'A2', 'a2': 'A2',
    'Пока А2, сдает 17 05 B1': 'A2',
    'окончание 13.06 курса на b1': 'A2', 'A1': 'A1', 'b2': 'B2',
    'Thorn-Prikker-Str. 30, Hagen, 58093': 'Not defined',
    'В2': 'B2', 'нулевой уровень, только пошел на курсы.': 'A0', 'ая в1': 'B1',
    'Ждет результат по B1': 'B1', 'А2( ждет итоги Б!)': 'A2',
    'b1 экзамен будет 12 апреля': 'B1','b1 (b2 ждет серт)': 'B1', 'С2': 'C2',
    'ждем B1': 'B1', 'Paderborn 33102, Schwabenweg 10': 'Not defined',
    'b1 (B2 ждет серт)': 'B1', 'Ждем B1 со дня на день': 'B1',
    'Б2 ( учит С1)': 'B2','B1 еще нет результата': 'A2',
    '31.05.2024': 'Not defined',
    'Lichtenfelser Straße 25, Untersiemau 96253': 'Not defined',
    'Учиться до сентября на B1': 'B1', 'b1 9ждет экзамен)': 'B1',
    'b1+': 'B1', 'гражданка': 'Not defined',
    'b1 (ждет результат)': 'B1', 'Б1 (учит Б2)': 'B1', 'б2+': 'B2',
    'Гражданин': 'B2','25 лет живет в Германии': 'C1', 'С1 -ая , Ня -а1': 'C1',
    'Ждем результат по B1': 'B1', 'b1 (b2 в июле экзамен)': 'B1',
    'Ждет со дня на день В1': 'B1',
    'А2 (В1 с 3 раза не сдала, бератер видела наши доки)': 'A2',
    'b1 (ждет результаты)': 'B1','А2 ( повтор на Б1)': 'A2',
    'B1, сдает B2 в апреле': 'B1', 'ждет сертификат B1': 'B1',
    'Б2( 16.02 экзамен С1)': 'B2', 'А1-А2': 'A1',
    'b1 ждет серт на днях на руки': 'B1',
    'b1 24 февраля экзамен, англ b2': 'B1',
    'А2 ( скоро екзамен)': 'A2', 'B1 (ждет результаты В2)': 'B1',
    'b1 (b2 15 марта экзамен)': 'B1', 'b2 (с1 экзамен 16 февраля)': 'B2',
    'Б1 ( ждет итог Б2)': 'B1','не сдавал, но гражданин': 'Not defined',
    'Нет сертификатов, но есть С1 англ, неоконченное высшее в ИТ (и еще одно высшее юридическое) , очень хочет в ИТ, сильно замотивирована именно н': 'Not defined',
    'А2, в процессе Б1': 'A2', 'A0': 'A0', 'А2(Б1 в марте экз)': 'A2',
    'учит A2': 'A2', 'Б1 ( проходит Б2)': 'B1', 'Б1 ( ждет итог )': 'B1',
    'НЯ - В1, АЯ - В1': 'B1','б1 (ждет рез-тат)': 'B1',
    'А2(ждет итоги Б1)': 'A2', 'в1-ня , в1-ая': 'B1',
    'ня-0, но англ B2+': 'A0', 'В': 'B1', 'будет B1 в июне': 'B1',
    'А2( включили нем в ангебот)': 'A2', 'а2-в1': 'A2', 'в2-с1': 'B2',
    'курс А2-В1 - сдача в июле, но вечерняя смена инт курсов, настроен получить гутшайн уже сейчас.': 'A2',
    'B1 (B2 должна до конца февраля получить)': 'B1',
    'b1 (b2 экзамен 6 февраля)': 'B1', 'A1-A2': 'A1',
    'Б1( может будет)': 'B1', 'А2 ( в процессе Б1)': 'A2',
    'b1 ждет результаты': 'B1', 'b1 ждет экзамен в феврале': 'B1',
    'В1, может уже В2?': 'B1',
    'A2 (идет доучивать В1 - 300 часов; предположительно до августа)': 'A2',
    'не учил': 'A0','Без 5 минут B1 (ждет результаты экзамена)': 'B1',
    'а1-а2 , ая свободный': 'A1', 'b2-c2': 'B2','а2, англ B1': 'A2',
    'А1': 'A1', 'А2 нем -В2 англ': 'A2', 'Проходит сейчас B1': 'B1',
    'Ждет результат по B1 в феврале': 'B1', 'Проходит сейчас повторно B1': 'B1',
    'b1 экзамен в феврале': 'B1',
    'Учиться на B1 во вторую смену, в первую хочет получить одобрение на обучение у нас': 'B1',
    'Б10Б2': 'B2', 'Б1?': 'B1', 'B1 есть, ждем B2 в конце месяца': 'B1',
    'B1-B2': 'B1','?': 'Not defined', 'b1 экзамен 26 января': 'A2',
    'А0': 'A0', 'а2 (б1 в сер января)': 'A2','f2': 'A2',
    'Учиться на B1': 'A2',
    'Сдала экзамен на B1, ждет в начале февраля результат': 'B1',
    'Сдавал 8 12 на B1 - ждет результат. 3 01 - аплейт - получил B1!': 'B1',
    'Б1-Б2': 'B1','б1 (до июля на В2)': 'B1',
    'А2 ( Б1 март )': 'A2', 'А2 (весной - еще 300 часов В1)': 'A2',
    'В январе будут результаты по экзамену на B1': 'B1',
    'б2 (с1 ждет рез-тат)': 'B2',
    'ня-0, ая-B1': 'A0', 'А2-Б1': 'A2',
    'B1 (почти, не сдала чуть) + англ В1': 'B1',
    'в1 ждем результаты': 'B1', 'А2 ( хочет просить совмещать)': 'A2',
    'B1 (ждет результаты)': 'B1', 'А2+': 'A2',
    'а2 (сдавала экз В1, но не сдала похоже)': 'A2', 'в1, идет на в2': 'B1',
    'b2-c1': 'B2', 'C1': 'C1', 'b1-b2': 'B1',
    'не учила ( разговорный) сразу пошла работать': 'B2',
    'Б1 ( проходит Б2 )': 'B1', 'a0-a1': 'A0',
    'Б1 ( был екзамен ждет итог )': 'B1',
    'Б2-С1': 'B2', 'b1 (учила, но не сдала В2)': 'B1',
    'ня а2, ая в1': 'A2',
    'A2 (идет на В1)': 'A2', 'B2-C2': 'B2',
    'немецкий - а1-а2, англ b1-b2': 'A2',
    'B2+': 'B2', 'в1, еще нет сертификата': 'B1', 'б1-б2': 'B1', 'Бй': 'B1',
    'ждет результаты по B1 экзамену': 'B1', 'b2 (ждет серт)': 'B2',
    'никакой': 'A0','в1 , хочет совмещать с в2': 'B1',
    90: 'Not defined', '.': 'Not defined', 'в1 (уже сдала В2)': 'B2',
    'b1 результат экзамена в феврале': 'B1',
    'в1 , экзамен на в2 15 декабря': 'B1', 'идет на А1': 'A1',
    'УТОЧНИТЬ!': 'Not defined','B2 (говорит без проблем - давно здесь)': 'B2',
    'B1 (до февраля)': 'B1','А2 ( Б2 в процессе)': 'B1', 'C': 'C1',
    'б1 заканчивает': 'B1', 'B1 (B2 экзамен в январе)': 'B1',
    '5 июля 2024 сдает экз на В2': 'B2', 'А2 (заканчив В1 в июне)': 'A2',
    'a2-б1': 'A2', 'В1?': 'B1',
    'b1 будет в январе экзамен, готов совмещать': 'B1',
    'b1 (b2 экзамен 2 марта)': 'B1-B2', 'B1 немецкий и английский Advance': 'B1',
    'A': 'A1', 'a2 (b1 экзамен 15 июня)': 'A2',
    'B2 (ждет итог экзамена)': 'B2',
    'b1 (b2 не сдал экзамен)': 'B1', 'В1 (учится на В2 до авг.': 'B1',
    'В2 - не сдал': 'B2', 'B2+ (не сдавал, но говорит)': 'B2',
    'b1 (ждет серт)': 'B1',
    'B1 вроде был (18 лет назад сдавал)': 'B1',
    'А2 (сдает B1 - 12 дек) - не сдал!': 'A2',
    'УТОЧНИТЬ': 'Not defined', 'b2 ждет серт': 'B2',
    'разговорный из украины, без сертификата': 'B2',
    'Ждет B1': 'B1', 'сдавала А2 в сентябре': 'A2',
    'В1, учится на В2 до няоб 24': 'B1',
    'Б1 ( ждет результат Б2)': 'B1',
    'точно уровень не знаю, но говорить могу - учила сама': 'B2',
    'А2-В1 учит': 'A2', 'В1 (учится на В2 уже)': 'B1',
    'В январе - В2 сдает': 'B2',
    'b1 должна получить результаты в феврале': 'B1'
}
deals['Level of Deutsch'] = deals['Level of Deutsch'].astype('object')\
    .replace(replace_dict).fillna("Not defined")
deals['Level of Deutsch'].unique()     

array(['Not defined', 'B1', 'A2', 'B2', 'А2-В1', 'A0', 'C2', 'C1', 'A1',
       'B1-B2'], dtype=object)

In [63]:
replace_double = {'А2-В1':'A2', 'B1-B2': 'B1'}
deals['Level of Deutsch'] = deals['Level of Deutsch']\
                              .replace(replace_double)
deals['Level of Deutsch'].unique()    

array(['Not defined', 'B1', 'A2', 'B2', 'A0', 'C2', 'C1', 'A1'],
      dtype=object)

In [64]:
deals['Level of Deutsch'].value_counts()

Level of Deutsch
Not defined    19797
B1              1072
B2               229
A2               218
C1                36
A1                28
A0                27
C2                 4
Name: count, dtype: int64

In [65]:
# Correct city replacement dictionary
replace_dict_city = {
    'Vor Ebersbach 1, 77761 Schiltach': 'Schiltach',
    'Poland , Gdansk , Al. Grunwaldzka 7, ap. 1a': 'Gdansk',
    'Karl-Liebknecht str. 24, Hildburghausen, Thüringen': 'Hildburghausen',
    '-': 'Unknown'
}

# Replace full address strings with city names and fill missing values
deals['City'] = deals['City'].replace(replace_dict_city).fillna("Not defined")
print(deals[['City', 'Level of Deutsch']].isna().sum())

City                0
Level of Deutsch    0
dtype: int64


In [66]:
# Course duration, Product, Deal Owner Name, Contact Name columns cleaning

In [67]:
deals['Course duration'].value_counts()

Course duration
11    3012
6      575
Name: count, dtype: Int64

In [68]:
# 1. Calculate the mode (most frequent value) of 'Course duration' for each 'Product'
mode_duration = deals.groupby('Product', observed=False)['Course duration'] \
    .agg(lambda x: x.mode()[0] if not x.mode().empty else None).reset_index()
mode_duration

,Product,Course duration
0,Data Analytics,<NA>
1,Digital Marketing,11
2,Find yourself in IT,<NA>
3,UX/UI Design,11
4,Web Developer,6


In [69]:
product_counts = deals['Product'].value_counts()
product_counts

Product
Digital Marketing      1990
UX/UI Design           1022
Web Developer           575
Find yourself in IT       4
Data Analytics            1
Name: count, dtype: int64

In [70]:
deals['Course duration'] = deals['Course duration'].fillna(0)
deals['Course duration'].value_counts()

Course duration
0     17824
11     3012
6       575
Name: count, dtype: Int64

In [71]:
deals.loc[(deals['Product'] == 'Web developer') & (deals['Course duration'] == 0), 'Course duration'] = 6
deals.loc[(deals['Product'] == 'UX/UI') & (deals['Course duration'] == 0), 'Course duration'] = 11
deals.loc[(deals['Product'] == 'Digital Marketing') & (deals['Course duration'] == 0), 'Course duration'] = 11

deals['Course duration'].value_counts()

Course duration
0     17824
11     3012
6       575
Name: count, dtype: Int64

In [72]:
deals['Product'] = deals['Product'].fillna('Not specified')

In [73]:
deals['Product'].value_counts()

Product
Not specified          17819
Digital Marketing       1990
UX/UI Design            1022
Web Developer            575
Find yourself in IT        4
Data Analytics             1
Name: count, dtype: int64

In [74]:
deals['Deal Owner Name'] = deals['Deal Owner Name'].fillna('Not Speicified')
deals['Deal Owner Name'].value_counts()

Deal Owner Name
Charlie Davis      2929
Julia Nelson       2196
Ulysses Adams      2150
Quincy Vincent     1878
Paula Underwood    1855
Ben Hall           1344
Nina Scott         1280
Victor Barnes      1229
Cara Iverson       1055
Diana Evans         998
Jane Smith          981
Rachel White        868
Kevin Parker        556
Ian Miller          489
Eva Kent            459
Bob Brown           327
Mason Roberts       268
Oliver Taylor       163
George King          94
Yara Edwards         84
Sam Young            66
Amy Green            63
Not Speicified       29
Alice Johnson        25
John Doe             19
Xander Dean           3
Wendy Clark           2
Zachary Foster        1
Name: count, dtype: int64

In [75]:
deals['Contact Name'] = deals['Contact Name'].fillna('Unknown')

In [76]:
# Checking if removing duplicates is still needed
deals[deals.duplicated()]

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Page,Campaign,Content,Term,...,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,Level of Deutsch,SLA in Seconds


In [77]:
deals.duplicated().sum()

np.int64(0)

In [78]:
#Calculate the mode (most frequent value) of Offer Total Amount for each Product
mode_amount = deals.groupby('Product', observed=False)['Offer Total Amount'] \
    .agg(lambda x: x.mode()[0] if not x.mode().empty else None).reset_index()
mode_amount

,Product,Offer Total Amount
0,Data Analytics,6000.0
1,Digital Marketing,11000.0
2,Find yourself in IT,0.0
3,Not specified,Unknown
4,UX/UI Design,11000.0
5,Web Developer,5000.0


In [79]:
# Group by 'Product' and calculate total and missing counts of 'Offer Total Amount'
offer_summary = deals.groupby('Product', observed=False).agg(
    total_rows=('Offer Total Amount', 'size'),                        # Total entries per product
    missing_rows=('Offer Total Amount', lambda x: x.isna().sum())     # Missing values per product
).reset_index()

# Sort by missing count for better readability
offer_summary = offer_summary.sort_values(by='missing_rows', ascending=False)

# Display the result
print(offer_summary)

               Product  total_rows  missing_rows
0       Data Analytics           1             0
1    Digital Marketing        1990             0
2  Find yourself in IT           4             0
3        Not specified       17819             0
4         UX/UI Design        1022             0
5        Web Developer         575             0


In [80]:
# Remove any currency symbols or commas and convert to numeric
deals['Offer Total Amount'] = pd.to_numeric(
    deals['Offer Total Amount'].replace(r'[\$,]', '', regex=True), 
    errors='coerce'
)
deals['Initial Amount Paid'] = pd.to_numeric(
    deals['Initial Amount Paid'].replace(r'[\$,]', '', regex=True), 
    errors='coerce'
)

# Define the condition where 'Initial Amount Paid' is greater than 'Offer Total Amount',
# and 'Offer Total Amount' is not zero
condition = (
    (deals['Offer Total Amount'] < deals['Initial Amount Paid']) & 
    (deals['Offer Total Amount'] != 0)
)

# Swap the values in the specified columns where the condition is True
deals.loc[condition, ['Offer Total Amount', 'Initial Amount Paid']] = \
    deals.loc[condition, ['Initial Amount Paid', 'Offer Total Amount']].values

# Display the updated DataFrame to confirm the changes
print(deals.head())

                    Id Deal Owner Name Closing Date            Quality  \
0  5805028000056864695        Ben Hall          NaT  E - Non Qualified   
1  5805028000056859489   Ulysses Adams          NaT  E - Non Qualified   
2  5805028000056832357   Ulysses Adams   2024-06-21     D - Non Target   
3  5805028000056824246        Eva Kent   2024-06-21  E - Non Qualified   
4  5805028000056873292        Ben Hall   2024-06-21     D - Non Target   

      Stage     Lost Reason       Page                  Campaign  \
0  New Lead             NaN  /eng/test             03.07.23women   
1  New Lead             NaN    /at-eng                       NaN   
2      Lost      Non target    /at-eng                engwien_AT   
3      Lost  Invalid number       /eng  04.07.23recentlymoved_DE   
4      Lost      Non target       /eng              discovery_DE   

              Content           Term  ... Education Type        Created Time  \
0                 v16          women  ...            NaN 2024-06-2

In [81]:
deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21411 entries, 0 to 21410
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Id                   21411 non-null  object        
 1   Deal Owner Name      21411 non-null  object        
 2   Closing Date         14495 non-null  datetime64[ns]
 3   Quality              21411 non-null  object        
 4   Stage                21411 non-null  object        
 5   Lost Reason          15942 non-null  object        
 6   Page                 21411 non-null  object        
 7   Campaign             16006 non-null  object        
 8   Content              14101 non-null  object        
 9   Term                 12424 non-null  object        
 10  Source               21411 non-null  object        
 11  Payment Type         496 non-null    object        
 12  Product              21411 non-null  object        
 13  Education Type       3299 non-n

In [82]:
# Create a new column that contains only the date part of 'Created Time' (no time)
deals['Created Date Only'] = deals['Created Time'].dt.date

# Compare only the dates (ignore time)
mistake_mask = deals['Closing Date'].dt.date < deals['Created Date Only']

# Filter rows where the closing date is earlier than the created date
mistakes = deals[mistake_mask]

# Display rows with detected mistakes
print(mistakes[['Created Time', 'Closing Date', 'Created Date Only']])
     

             Created Time Closing Date Created Date Only
446   2024-06-16 00:06:00   2024-06-11        2024-06-16
2035  2024-05-25 21:29:00   2024-05-22        2024-05-25
2728  2024-05-12 11:19:00   2024-05-07        2024-05-12
2958  2024-05-08 15:31:00   2024-05-07        2024-05-08
2969  2024-05-08 12:54:00   2024-05-07        2024-05-08
3624  2024-04-30 15:16:00   2024-04-23        2024-04-30
4030  2024-04-24 17:30:00   2024-04-17        2024-04-24
4089  2024-04-23 21:44:00   2024-04-18        2024-04-23
4349  2024-04-21 08:57:00   2024-04-18        2024-04-21
4435  2024-04-20 06:19:00   2023-08-21        2024-04-20
4705  2024-04-17 09:10:00   2024-04-12        2024-04-17
4810  2024-04-16 19:30:00   2024-04-11        2024-04-16
4900  2024-04-16 15:35:00   2024-04-10        2024-04-16
4987  2024-04-16 07:50:00   2024-04-11        2024-04-16
5385  2024-04-13 04:14:00   2024-04-08        2024-04-13
5549  2024-04-12 01:40:00   2024-04-09        2024-04-12
6093  2024-04-07 15:02:00   202

In [83]:
# Create a mask where the closing date is earlier than the created date (ignoring time)
mistake_mask = deals['Closing Date'].dt.date < deals['Created Date Only']

# Swap 'Created Time' and 'Closing Date' where the mistake occurs
deals.loc[mistake_mask, ['Created Time', 'Closing Date']] = deals.loc[mistake_mask, ['Closing Date', 'Created Time']].values

#  Check the rows where the swap happened to confirm
print(deals.loc[mistake_mask, ['Created Time', 'Closing Date']])

      Created Time        Closing Date
446     2024-06-11 2024-06-16 00:06:00
2035    2024-05-22 2024-05-25 21:29:00
2728    2024-05-07 2024-05-12 11:19:00
2958    2024-05-07 2024-05-08 15:31:00
2969    2024-05-07 2024-05-08 12:54:00
3624    2024-04-23 2024-04-30 15:16:00
4030    2024-04-17 2024-04-24 17:30:00
4089    2024-04-18 2024-04-23 21:44:00
4349    2024-04-18 2024-04-21 08:57:00
4435    2023-08-21 2024-04-20 06:19:00
4705    2024-04-12 2024-04-17 09:10:00
4810    2024-04-11 2024-04-16 19:30:00
4900    2024-04-10 2024-04-16 15:35:00
4987    2024-04-11 2024-04-16 07:50:00
5385    2024-04-08 2024-04-13 04:14:00
5549    2024-04-09 2024-04-12 01:40:00
6093    2024-04-03 2024-04-07 15:02:00
6237    2023-10-03 2024-04-05 10:40:00
7082    2024-03-19 2024-03-26 16:44:00
7477    2024-03-13 2024-03-20 12:37:00
8437    2024-02-28 2024-03-06 09:15:00
8593    2024-02-29 2024-03-04 08:17:00
8819    2024-02-22 2024-02-29 08:54:00
8859    2024-02-21 2024-02-28 14:33:00
9116    2024-02-24 2024-0

In [84]:
deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21411 entries, 0 to 21410
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Id                   21411 non-null  object        
 1   Deal Owner Name      21411 non-null  object        
 2   Closing Date         14495 non-null  datetime64[ns]
 3   Quality              21411 non-null  object        
 4   Stage                21411 non-null  object        
 5   Lost Reason          15942 non-null  object        
 6   Page                 21411 non-null  object        
 7   Campaign             16006 non-null  object        
 8   Content              14101 non-null  object        
 9   Term                 12424 non-null  object        
 10  Source               21411 non-null  object        
 11  Payment Type         496 non-null    object        
 12  Product              21411 non-null  object        
 13  Education Type       3299 non-n

In [85]:
# Drop the helper column after the correction
deals.drop(columns='Created Date Only', inplace=True)

In [86]:
deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21411 entries, 0 to 21410
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Id                   21411 non-null  object        
 1   Deal Owner Name      21411 non-null  object        
 2   Closing Date         14495 non-null  datetime64[ns]
 3   Quality              21411 non-null  object        
 4   Stage                21411 non-null  object        
 5   Lost Reason          15942 non-null  object        
 6   Page                 21411 non-null  object        
 7   Campaign             16006 non-null  object        
 8   Content              14101 non-null  object        
 9   Term                 12424 non-null  object        
 10  Source               21411 non-null  object        
 11  Payment Type         496 non-null    object        
 12  Product              21411 non-null  object        
 13  Education Type       3299 non-n

In [87]:
# Replace incorrect 'Lost' status with 'Payment Done' where payment has been made
deals.loc[deals['Initial Amount Paid'] > 0, 'Stage'] = 'Payment Done'

# Convert 'Lost Reason' to categorical if it's not already
if not isinstance(deals['Lost Reason'].dtype, pd.CategoricalDtype):
    deals['Lost Reason'] = deals['Lost Reason'].astype('category')

# Add 'No reason' to the list of categories if it's not already included
if 'No reason' not in deals['Lost Reason'].cat.categories:
    deals['Lost Reason'] = deals['Lost Reason'].cat.add_categories(['No reason'])

# Set 'Lost Reason' to 'No reason' where payment has been made
deals.loc[deals['Initial Amount Paid'] > 0, 'Lost Reason'] = 'No reason'

mask = deals['Initial Amount Paid'] > 0

# Number of updated rows
num_updated = mask.sum()

print("Number of updated rows:", num_updated)

Number of updated rows: 3288


In [88]:
# Convert the data types of certain columns to categorical
category_columns = ['Deal Owner Name', 'Quality', 'Stage',
                    'Page', 'Campaign', 'Content',
                     'Term', 'Source', 'Payment Type', 'Product', 'City',
                     'Level of Deutsch', 'Education Type']
for col in category_columns:
  deals[col] = deals[col].astype('category')
deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21411 entries, 0 to 21410
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Id                   21411 non-null  object        
 1   Deal Owner Name      21411 non-null  category      
 2   Closing Date         14495 non-null  datetime64[ns]
 3   Quality              21411 non-null  category      
 4   Stage                21411 non-null  category      
 5   Lost Reason          17324 non-null  category      
 6   Page                 21411 non-null  category      
 7   Campaign             16006 non-null  category      
 8   Content              14101 non-null  category      
 9   Term                 12424 non-null  category      
 10  Source               21411 non-null  category      
 11  Payment Type         496 non-null    category      
 12  Product              21411 non-null  category      
 13  Education Type       3299 non-n

In [89]:
# Count missing values before filling
missing_before = deals['Education Type'].isna().sum()

# Compute the most frequent 'Education Type' per group, with observed=True
mode_education = deals.groupby(
    ['Offer Total Amount', 'Course duration', 'Product'],
    observed=True  # Explicitly set to future default
)['Education Type'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else None
).reset_index()

# Merge the mode data back into the original DataFrame
deals = deals.merge(
    mode_education,
    on=['Offer Total Amount', 'Course duration', 'Product'],
    how='left',
    suffixes=('', '_mode')
)

# Fill missing values with the calculated mode
deals['Education Type'] = deals['Education Type'].fillna(deals['Education Type_mode'])

# Count missing values after filling
missing_after = deals['Education Type'].isna().sum()
filled_count = missing_before - missing_after

# Drop helper column
deals.drop(columns=['Education Type_mode'], inplace=True)

# Show results
print(f"Missing values before filling: {missing_before}")
print(f"Missing values after filling: {missing_after}")
print(f"Filled values: {filled_count}")

Missing values before filling: 18112
Missing values after filling: 17971
Filled values: 141


In [90]:
# Define a function to classify the Payment type based on deal information
def classify_payment_type(row):
    # If the existing value is already 'Reservation', keep it as is
    if row['Payment Type'] == 'Reservation':
        return 'Reservation'
    
    # If both the offer and the initial payment are zero, no payment has occurred
    elif row['Offer Total Amount'] == 0 and row['Initial Amount Paid'] == 0:
        return 'No Payments'
    
    # If there's an offer amount but nothing was paid yet
    elif row['Initial Amount Paid'] == 0:
        return 'No Initial Payments'
    
    # If the absolute difference between the offer and initial payment is greater than 200,
    # this likely means the payment was made in installments or involves recurring charges
    elif abs(row['Offer Total Amount'] - row['Initial Amount Paid']) > 200:
        return 'Recurring Payments'
    
    # Otherwise, we assume it was a full one-time payment
    else:
        return 'One Payment'

# Apply the classification function to each row in the DataFrame
# axis=1 ensures that the function receives a full row (Series) for evaluation
deals['Payment Type'] = deals.apply(classify_payment_type, axis=1)
deals['Payment Type'].value_counts()

Payment Type
One Payment            17375
Recurring Payments      3164
No Payments              825
No Initial Payments       42
Reservation                5
Name: count, dtype: int64

In [91]:
# Fill missing values in 'Education Type'
# First, check if the column is categorical and add 'Unknown' if needed
if isinstance(deals['Education Type'].dtype, pd.CategoricalDtype):
    if 'Unknown' not in deals['Education Type'].cat.categories:
        deals['Education Type'] = deals['Education Type'].cat.add_categories(['Unknown'])

# Count how many missing before filling
education_na_count = deals['Education Type'].isna().sum()
deals['Education Type'] = deals['Education Type'].fillna('Unknown')

# Fill missing values in 'Lost Reason'
if isinstance(deals['Lost Reason'].dtype, pd.CategoricalDtype):
    if 'Not lost' not in deals['Lost Reason'].cat.categories:
        deals['Lost Reason'] = deals['Lost Reason'].cat.add_categories(['Not lost'])

lost_reason_na_count = deals['Lost Reason'].isna().sum()
deals['Lost Reason'] = deals['Lost Reason'].fillna('Not lost')

# Fill missing values in 'SLA in Seconds' with 0
sla_na_count = deals['SLA in Seconds'].isna().sum()
deals['SLA in Seconds'] = deals['SLA in Seconds'].fillna(0)

# Print how many missing values were filled
print(f"Filled 'Education Type': {education_na_count} values")
print(f"Filled 'Lost Reason': {lost_reason_na_count} values")
print(f"Filled 'SLA in Seconds': {sla_na_count} values")

print(deals['Education Type'].value_counts())
print(deals['Lost Reason'].value_counts())
print(deals['SLA in Seconds'].value_counts())

Filled 'Education Type': 17971 values
Filled 'Lost Reason': 4087 values
Filled 'SLA in Seconds': 5907 values
Education Type
Unknown    17971
Morning     3028
Evening      412
Name: count, dtype: int64
Lost Reason
Not lost                                   4087
Doesn't Answer                             3972
No reason                                  3288
Changed Decision                           1654
Non target                                 1635
Duplicate                                  1541
Invalid number                             1444
Stopped Answering                          1083
Expensive                                   584
needs time to think                         510
Conditions are not suitable                 485
Next stream                                 193
Inadequate                                  166
Not for myself                              141
Does not speak English                      138
Didn't leave an application                 133
Considering a diffe

In [92]:
# Drop the 'Content' column from the DataFrame if it's no longer needed
deals = deals.drop(columns=['Content'])

# Drop the 'Page' column — for example, if it contains redundant or irrelevant tracking data
deals = deals.drop(columns=['Page'])

# Drop the 'Term' column, likely related to search keywords or UTM parameters that aren't needed
deals = deals.drop(columns=['Term'])
deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21411 entries, 0 to 21410
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Id                   21411 non-null  object        
 1   Deal Owner Name      21411 non-null  category      
 2   Closing Date         14495 non-null  datetime64[ns]
 3   Quality              21411 non-null  category      
 4   Stage                21411 non-null  category      
 5   Lost Reason          21411 non-null  category      
 6   Campaign             16006 non-null  category      
 7   Source               21411 non-null  category      
 8   Payment Type         21411 non-null  object        
 9   Product              21411 non-null  category      
 10  Education Type       21411 non-null  category      
 11  Created Time         21411 non-null  datetime64[ns]
 12  Course duration      21411 non-null  Int8          
 13  Months of study      840 non-nu

In [93]:
# Saving the updated file after cleaning
deals.to_excel("Deals (Final).xlsx", index=False)